In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# 输入研究区的shp文件所在路径
wuhan_shp ="D:/NDVI_exercise/wuhan/wuhanregion.shpe"
# shp 转 ee 
wuhan_ee = geemap.shp_to_ee(wuhan_shp)
# 添加到layer
# Map.addLayer(wuhan_ee,{},"wuhan")
Map.centerObject(wuhan_ee,7)   # 设置以研究区居中

In [4]:
permanent_waterArea = []
accuracy = []
kappa = []

# landsat 7 可用年份为1999-2020

In [5]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

# 可视化参数设置
vis_params = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}
# NDVI and mNDWI 函数的定义
def NDVI(image):
    ndvi=image.normalizedDifference(['B4', 'B3']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B2', 'B5']).rename("mNDWI")
    return image.addBands(mndwi)

# RandomForest预测使用的波段
bands7 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
# 分类标签
label = 'waterclass'

In [6]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(wuhan_ee).select('constant').rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")

In [7]:
# 定义的函数
def imageSample(image):
    # Overlay the points on the imagery to get training.
    sample_point = image.select(bands7).sampleRegions(**{
        'collection': trainingPartition,
        'properties': ['jrcwaterclass'],
        'scale': 30
    })
    return sample_point


#对Landsat-7进行分类
def training(image):
    return image.clip(wuhan_ee).select(bands7).classify(trainedClassifier).eq(1).selfMask().select('classification').rename('waterclass')

In [10]:
for year in range(2010,2014,1):
    landsat7_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
        .filterBounds(wuhan_ee) \
        .filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)) \
        .map(cloudMaskL457)
    landsat_ndvi_mndwi = landsat7_images.map(NDVI).map(mNDWI)
    # 计算有效像元的个数
    validPixel = landsat7_images.count().select('B3').clip(wuhan_ee).rename('count')
    JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/{}".format(year)).clip(wuhan_ee)
    JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
    JRC_samplepoints = JRC_permanentwater.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    # 70% 做训练点位，30% 做验证点位
    withRandom = JRC_samplepoints.randomColumn('random')
    #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
    split = 0.7
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))
    points_collection = landsat_ndvi_mndwi.map(imageSample).flatten()
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,'jrcwaterclass',bands7)
    landsatImage_classify = landsat_ndvi_mndwi.map(training)
    waterpixel_count = landsatImage_classify.sum()
    wuhan_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).select('waterclass').rename('frequency')
    wuhan_permanentwater = wuhan_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
    waterarea = wuhan_permanentwater.multiply(ee.Image.pixelArea()).divide(1e6)
    areas = waterarea.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': wuhan_ee.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    print('the area of permanent water in {}year:{}km²'.format(year,areas.get('waterclass').getInfo()))
    permanent_waterArea.append(areas.get('waterclass').getInfo())
    imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
    permanent_water_layer = imagecollection.sum()
    #验证
    # Overlay the points on the imagery to get training.
    JRC_sample_point = permanent_water_layer.sampleRegions(**{
        'collection': testingPartition,
        'properties': ['jrcwaterclass'],
        'scale': 30
    })
    # 利用误差矩阵进行验证
    test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
    accuracy.append(test_accuracy.accuracy().getInfo())
    kappa.append(test_accuracy.kappa().getInfo())
    print('total caccuracy in {} year:{}'.format(year,test_accuracy.accuracy().getInfo()))
    print('kappa in {} year:{}'.format(year,test_accuracy.kappa().getInfo()))

the area of permanent water in 2010year:534.8761892960863km²
total caccuracy in 2010 year:0.9126760563380282
kappa in 2010 year:0.7810168142473386
the area of permanent water in 2011year:555.3892211943136km²
total caccuracy in 2011 year:0.9054054054054054
kappa in 2011 year:0.7742407865834524
the area of permanent water in 2012year:576.5749656876685km²
total caccuracy in 2012 year:0.9080779944289693
kappa in 2012 year:0.7660591220552515
the area of permanent water in 2013year:588.874673597684km²
total caccuracy in 2013 year:0.9317507418397626
kappa in 2013 year:0.8464144886757683


# landsat 8 可用年份为2014-2020

In [11]:
# 去云掩膜函数
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)
# landsat8 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}
# JRC数据集的可视化参数
visualization = {
    'bands': ['waterClass'],
    'min': 0.0,
    'max': 3.0,
    'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
# NDVI and mNDWI 函数的定义
def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

# RandomForest预测使用的波段
bands8 = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'


In [12]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(wuhan_ee).select('constant').rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")

In [13]:
# 定义的函数
def imageSample(image):
    # Overlay the points on the imagery to get training.
    sample_point = image.select(bands8).sampleRegions(**{
        'collection': trainingPartition,
        'properties': ['jrcwaterclass'],
        'scale': 30
    })
    return sample_point


#对Landsat-7进行分类
def training(image):
    return image.clip(wuhan_ee).select(bands8).classify(trainedClassifier).eq(1).selfMask().select('classification').rename('waterclass')

In [19]:
for year in range(2014,2019,1):
    landsat8_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(wuhan_ee) \
        .filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)) \
        .map(maskL8sr)
    landsat_ndvi_mndwi = landsat8_images.map(NDVI).map(mNDWI)
    # 计算有效像元的个数
    validPixel = landsat8_images.count().select('B3').clip(wuhan_ee).rename('count')
    JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/{}".format(year)).clip(wuhan_ee)
    JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
    JRC_samplepoints = JRC_permanentwater.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    # 70% 做训练点位，30% 做验证点位
    withRandom = JRC_samplepoints.randomColumn('random')
    #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
    split = 0.7
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))
    points_collection = landsat_ndvi_mndwi.map(imageSample).flatten()
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,'jrcwaterclass',bands8)
    landsatImage_classify = landsat_ndvi_mndwi.map(training)
    waterpixel_count = landsatImage_classify.sum()
    wuhan_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).select('waterclass').rename('frequency')
    wuhan_permanentwater = wuhan_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
    waterarea = wuhan_permanentwater.multiply(ee.Image.pixelArea()).divide(1e6)
    areas = waterarea.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': wuhan_ee.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    print('the area of permanent water in {}year:{}km²'.format(year,areas.get('waterclass').getInfo()))
    permanent_waterArea.append(areas.get('waterclass').getInfo())
    imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
    permanent_water_layer = imagecollection.sum()
    #验证
    # Overlay the points on the imagery to get training.
    JRC_sample_point = permanent_water_layer.sampleRegions(**{
        'collection': testingPartition,
        'properties': ['jrcwaterclass'],
        'scale': 30
    })
    # 利用误差矩阵进行验证
    test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
    accuracy.append(test_accuracy.accuracy().getInfo())
    kappa.append(test_accuracy.kappa().getInfo())
    print('total caccuracy in {} year:{}'.format(year,test_accuracy.accuracy().getInfo()))
    print('kappa in {} year:{}'.format(year,test_accuracy.kappa().getInfo()))

the area of permanent water in 2014year:556.6835433897309km²
total caccuracy in 2014 year:0.9181286549707602
kappa in 2014 year:0.8064516129032259
the area of permanent water in 2015year:605.1810515257071km²
total caccuracy in 2015 year:0.9093484419263456
kappa in 2015 year:0.7980476990739085


ProxyError: ('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response'))

# 结果

In [ ]:
print(permanent_waterArea)

In [ ]:
print(accuracy)

In [ ]:
print(kappa)